# Hidden Markov Models

This notebook demonstrates a basic HMM to model the Eisner task: https://www.cs.jhu.edu/~jason/papers/eisner.tnlp02.pdf

In [26]:
import numpy as np


class HMM():
    def __init__(self, pi, A, B):
        """Initializes an HMM initial state parameters, transition probabilities, and
        observation probabilities.
        
        Parameters
        ----------
        pi : numpy array, shape = [n_states]
             Initial state probabilities.
        A : numpy array, shape = [n_states, n_states]
            Transition matrix.
        B : numpy array, shape = [n_states, n_observations]
            Emission prbability matrix.
        """
        self.pi_ = pi
        self.A_ = A
        self.B_ = B
        self.n_states_ = A.shape[0]
        
    def forward(self, O):
        """Computes the likelihood of a given observation sequence.
        
        Parameters
        ----------
        O : numpy array, shape = [seq_length]
        """
        seq_length = O.shape[0]
        probs = np.zeros((self.n_states_, seq_length))
        
        # Initialization
        for i in range(self.n_states_):
            probs[i, 0] = self.pi_[i] * self.B_[i, O[0]]
            
        # Recursive step
        for t in range(1, seq_length):
            for s in range(self.n_states_):
                for sp in range(self.n_states_):
                    probs[s, t] += probs[sp, t-1] * self.A_[sp, s] * self.B_[s, O[t]]
                    
        print(probs)
                    
        # Termination step
        probs = np.sum(probs[:, -1])
        
        return probs
    
    def viterbi(self, O):
        """Computes the most likely state sequence given an observation sequence.
        
        Parameters
        ----------
        O : numpy array, shape = [seq_length]
        """
        seq_length = O.shape[0]
        probs = np.zeros((self.n_states_, seq_length))
        backpointer = np.zeros((self.n_states_, seq_length))
        
        # Initialization
        for s in range(self.n_states_):
            probs[s, 0] = self.pi_[s] * self.B_[s, O[0]]
            backpointer[s, 0] = 0
            
        # Recursive step
        for t in range(1, seq_length):
            for s in range(self.n_states_):
                temp = np.zeros((self.n_states_))
                for sp in range(self.n_states_):
                    temp[sp] = probs[sp, t-1] * self.A_[sp, s] * self.B_[s, O[t]]
                    
                probs[s, t] = np.max(temp)
                backpointer[s, t] = np.argmax(temp)
                
        # Termination step
        best_path_prob = np.max(probs[:, -1])
        best_path_pointer = np.argmax(probs[:, -1])
        
        print(probs)
        
        bestpath = np.zeros((seq_length), dtype=np.int32)
        bestpath[-1] = best_path_pointer
        
        for i in range(seq_length-2, -1, -1):
            print(i)
            bestpath[i] = backpointer[bestpath[i+1], i]
                    
        return bestpath, best_path_prob

In [27]:
pi = np.array([.2, .8])

A = np.array([
    [.5, .5],
    [.4, .6]
])

B = np.array([
    [.5, .4, .1],
    [.2, .4, .4]
])

model = HMM(pi, A, B)
seq = np.array([2, 0, 2])
print(model.forward(seq))

print(model.viterbi(seq))

[[0.02     0.069    0.005066]
 [0.32     0.0404   0.023496]]
0.02856200000000001
[[0.02   0.064  0.0032]
 [0.32   0.0384 0.0128]]
1
0
(array([0, 1, 1], dtype=int32), 0.012800000000000004)
